<a href="https://colab.research.google.com/github/st185229/UEL-CN-7030---Machine-Learning-on-Big-Data-This/blob/main/Copy_of_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.kaggle.com/code/sawsanshakir/stress-detection-using-pyspark/notebook

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('task1data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'spark-3.1.1-bin-hadoop3.2.tgz'
Access is denied.


Exception: Unable to find py4j in /content/spark-3.1.1-bin-hadoop3.2\python, your SPARK_HOME may not be configured correctly

In [ ]:
from pyspark.sql.types import StructType,StructField,DoubleType
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc =SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

data = sqlContext.read.format('com.databricks.spark.csv').option("escape", "\"").options(header='true', inferschema='true').load('task1data/train.csv')
data.show(2)

C:\Users\sthoe09258\AppData\Roaming\JetBrains\DataSpell2022.1\projects\workspace\venv\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------+---------+--------+--------------+--------------------+-----+-----+----------+----------------+------------+-----------+-----------+-----------------+--------------+------------------+-------------+------------+---------------+------------+-----------------+----------------+--------------+----------+-----------+------------+--------------+-------------+--------------+----------------+-------------+----------------+---------------+-------------+---------------+-------------+------------+----------------+-----------------+---------------+--------------+---------------+---------------+---------------+------------+--------------+------------+---------------+---------------+---------------+---------------+-------------+----------------+----------------+--------------+----------------+---------------+----------------+---------------+----------------+------------+-------------+-------------+------------+-------------+---------------+---------------+---------------+---------------+--

In [ ]:
col_list = ['post_id0', 'text', 'label']
data = data.select([column for column in data.columns if column in col_list])

In [ ]:
data.show(10)

+--------+--------------------+-----+
|post_id0|                text|label|
+--------+--------------------+-----+
|    2092|Hello, You are in...|    0|
|    1567|I want to know ab...|    0|
|    1550|Have any of you r...|    0|
|    2785|Whenever I have a...|    1|
|    1936|Throwaway because...|    1|
|    1125|But I really do e...|    1|
|     945|We are 21 now, bo...|    0|
|    1811|I have started wo...|    0|
|    2671|  ---   **tl;dr**...|    0|
|    1463|I just feel so na...|    1|
+--------+--------------------+-----+
only showing top 10 rows



In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkContext

sc_1 =SparkContext.getOrCreate()
sqlContext_1 = SQLContext(sc_1)

data_test = sqlContext.read.format('com.databricks.spark.csv').option("escape", "\"").options(header='true', inferschema='true').load('task1data/test.csv')

In [ ]:
data_test.show(1)

+--------+---------+--------+--------------+--------------------+----+----------+----------------+------------+-----------+-----------+-----------------+--------------+------------------+-------------+------------+---------------+------------+-----------------+----------------+--------------+----------+-----------+------------+--------------+-------------+--------------+----------------+-------------+----------------+---------------+-------------+---------------+-------------+------------+----------------+-----------------+---------------+--------------+---------------+---------------+---------------+------------+--------------+------------+---------------+---------------+---------------+---------------+-------------+----------------+----------------+--------------+----------------+---------------+----------------+---------------+----------------+------------+-------------+-------------+------------+-------------+---------------+---------------+---------------+---------------+---------

In [ ]:
col_list = ['post_id0', 'text']
data_test = data_test.select([column for column in data_test.columns if column in col_list])

In [ ]:
data_test.show(10)

+--------+--------------------+
|post_id0|                text|
+--------+--------------------+
|     415|For anxiety/depre...|
|    2691|His response was ...|
|    1322|He would call me ...|
|    2272|Back ground, me a...|
|    1967|Sally is around t...|
|     354|Eat probiotic foo...|
|     965|I obsess about th...|
|    1600|Hi! This is my fi...|
|     527|already a few mon...|
|    1771|Hi everyone, I'm ...|
+--------+--------------------+
only showing top 10 rows



In [ ]:
import gensim.parsing.preprocessing as gsp
from pyspark.sql.types import StringType
from gensim import utils


filters = [
    gsp.strip_tags,
    gsp.strip_punctuation,
    gsp.strip_multiple_whitespaces,
    gsp.strip_numeric,
    gsp.remove_stopwords,
    gsp.strip_short,
    gsp.stem_text
]

def clean_text(x):
    s = x[1]
    s = s.lower()
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return (x[0],s,x[2])

In [ ]:
input_rdd = data.rdd.map(lambda x : clean_text(x))

#input_df  = input_rdd.toDF(['post_id0', 'text', 'label'])
#input_df .show(10)

In [ ]:

input_df = input_rdd.toDF()
input_df

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 35.0 failed 1 times, most recent failure: Lost task 0.0 in stage 35.0 (TID 35) (AP-3mc9nGWEKZtl.int.elekta.com executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org

In [ ]:
def test_clean_text(x):
    s = x[1]
    s = s.lower()
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return (x[0],s)


input_rdd_test = data_test.rdd.map(lambda x : test_clean_text(x))

test_input_df  = input_rdd_test.toDF(['post_id0', 'text'])
test_input_df .show(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 36.0 failed 1 times, most recent failure: Lost task 0.0 in stage 36.0 (TID 36) (AP-3mc9nGWEKZtl.int.elekta.com executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org